In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
import random
import numpy as np

#Defining parameters
epsilon = 0.02
gamma = 50
M = 1e-3
maxh = 0.01 
order = 1 
tau = 0.1            
time = 0.0
T = 1


#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
V = H1(mesh, order=1) 

fes = V*V #FESpace([V,V])

phi1,mu1 = fes.TrialFunction()  #n+1
v,w = fes.TestFunction()

phi_old = GridFunction(V)
#Initialising phi_0 random values between -1 and 1
random_values = np.random.uniform(-1, 1, V.ndof)
phi_old.vec.FV().NumPy()[:] = random_values


#f=LinearForm(fes)
#f += tau*rhs_1*v*dx+gamma*(phi_old**3-phi_old)*w*dx+ phi_old*v*dx

A = BilinearForm(fes)
A += phi1*v*dx+tau*M*grad(mu1)*grad(v)*dx + mu1*w*dx-epsilon*grad(phi1)*grad(w)*dx
A.Assemble()

t = 0

gfu = GridFunction(fes)

gfut = GridFunction(gfu.space, multidim = 0)
gfut.AddMultiDimComponent(gfu.vec)


while t<T:
    t+=tau
    rhs = exp(-t)*x
    
    f=LinearForm(fes)
    f+= tau*rhs*v*dx+gamma*(phi_old**3-phi_old)*w*dx+ phi_old*v*dx
    f.Assemble()

    gfu.vec.data = A.mat.Inverse(freedofs = fes.FreeDofs())*f.vec

    gfut.AddMultiDimComponent(gfu.vec)
    phi_old.vec.data = gfu.components[0].vec

    
    



In [2]:
Draw(gfut.components[0], mesh, interpolate_multidim = True, animate = True)
#Draw(gfu.components[0])

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Fully explicit scheme

Discretize $\partial_t \phi = \frac{\phi^{n+1}-\phi^n}{\tau}$, $\mu = \mu^{n+1}$, $\phi = \phi^n$ and $\nabla \phi = \nabla \phi^{n+1}$.

$$
\begin{aligned}
(\frac{\phi^{n+1}-\phi^n}{\tau},v) + (M\nabla \mu^{n+1}, \nabla v) &= 0\ \ \ \ \forall v \in \mathbb{V} \\
(\mu^{n+1},w)-(\epsilon \nabla \phi^{n+1}, \nabla w) 
&= (\gamma ((\phi^n)^3 -\phi^n),w) \ \ \ \  \forall w \in \mathbb{V} 
\end{aligned}
$$


In [2]:
def FullyExplicitScheme(V,v,w,mu,phi,phi_old,phi_trial,mu_trial):

    #Storing solutions for each timestep
    phi_solutions = []
    mu_solutions = []

    #Bilinear and linear form for the first equation
    A_phi = BilinearForm(V)
    A_phi += (phi_trial * v) * dx + tau * M * grad(mu_trial) * grad(v) * dx  
    A_phi.Assemble()

    f_phi = LinearForm(V)

    #Bilinear and linear form for the second equation
    A_mu = BilinearForm(V)
    A_mu += (mu_trial * w) * dx - epsilon * grad(phi_trial) * grad(w) * dx
    A_mu.Assemble()

    f_mu = LinearForm(V)

    t = 0
    while t < T:
        t += tau

        rhs_phi = M*(3*x**2-3)
        rhs_mu= (x**3-3*x)-gamma*(exp(-3*t)*(x**6+3*x**4+3*x**2+1)-exp(-t)*(x**2+1))-epsilon*(2*exp(-t)*x)


        #Right hand side of first equation
        f_phi = LinearForm(V)
        f_phi += phi_old * v * dx  + tau*rhs_phi*v*dx # er bare 0 her originalt men definerer uansett fordi vi splitter opp diskretiserignen for phi, skriv ned
        f_phi.Assemble()

        #Solve for phi^{n+1}
        phi = GridFunction(V,name="uDG")
        phi.vec.data = A_phi.mat.Inverse(V.FreeDofs()) * f_phi.vec
        

        #Right hand side of second equation
        phi_nonlinear = CoefficientFunction(gamma * (phi_old**3 - phi_old))
        f_mu = LinearForm(V)  
        f_mu += phi_nonlinear * w * dx + tau*rhs_mu*w*dx
        f_mu.Assemble()

        #Solve for mu^{n+1}
        mu = GridFunction(V,name="uDG")
        mu.vec.data += A_mu.mat.Inverse(V.FreeDofs()) * f_mu.vec

        
        #phi_solutions.append(phi.vec.FV().NumPy().copy())
        #mu_solutions.append(mu.vec.FV().NumPy().copy())

        phi_solutions.append(phi)
        mu_solutions.append(mu)

        #phi_solutions.append(phi)
        #mu_solutions.append(mu) 
        phi_old.vec.data = phi.vec
        #print(mu.vec.FV().NumPy().copy())

    return phi_solutions, mu_solutions


In [3]:
phi_solutions, mu_solutions = FullyExplicitScheme(V,v,w,mu,phi,phi_old,phi_trial,mu_trial)

In [4]:
phi = phi_solutions[-1]
mu = mu_solutions[-1]


In [5]:
def phi_exact(t):
    return exp(-t)*(x**2+1)

mu_exact = x**3-3*x


In [6]:
e_h = mu_exact-mu

norm = np.sqrt(Integrate(e_h*e_h,mesh))
norm

386959.4199863015

Kilde:https://docu.ngsolve.org/nightly/i-tutorials/unit-3.1-parabolic/parabolic.html

https://docu.ngsolve.org/latest/whetting_the_appetite/navierstokes.html